In [40]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt    
import plotly.express as px
from datetime import datetime, timedelta
import plotly.graph_objects as go



In [41]:
weekly_path = '/Users/Marcy_Student/Desktop/Exercises/Python/MOD5PROJECT/data/LinkNYC_Weekly_Usage_cleaned_20251008.csv'

In [42]:
df = pd.read_csv(weekly_path)

In [43]:
df.head()

,Unnamed: 0,report_ending_weekly_starting,number_of_sessions,average_session_length,number_of_unique_clients,tb_downloaded,tb_uploaded,cumulative_bandwidth,cumulative_sessions,cumulative_subscribers,sessions_per_user,GB_per_session,log_number_of_sessions,log_number_of_unique_clients,log_tb_downloaded,month
0,0,2020-01-05,23086410,00:04:07,683250,121.01,12.67,14353.83,2251215052,8461839,33.789111,0.005790,16.954755,13.434616,4.795873,1
1,1,2020-01-12,23584159,00:04:07,640118,116.52,11.14,14501.59,2284581863,8502485,36.843455,0.005413,16.976086,13.369408,4.758063,1
2,2,2020-01-19,22004022,00:04:07,592715,114.50,10.52,14626.62,2306585885,8536567,37.124119,0.005682,16.906736,13.292469,4.740575,1
3,3,2020-01-26,20245042,00:04:15,602685,118.05,10.13,14754.79,2326830927,8571523,33.591415,0.006331,16.823420,13.309150,4.771108,1
4,4,2020-02-02,22356460,00:04:11,617337,120.69,10.94,14886.42,2349187387,8615640,36.214353,0.005888,16.922626,13.333170,4.793225,2


In [44]:
df = df.drop('Unnamed: 0', axis=1)

In [45]:
# Identify the date column 'report_ending_weekly_starting'
# change the date colum to datetime format
# Use conservative parsing with errors='coerce' so bad rows become NaT
date_col = None
for c in df.columns:
    if 'week' in c.lower() or 'date' in c.lower():
        date_col = c
        break

df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
df = df.sort_values(date_col).reset_index(drop=True)
df.head()

,report_ending_weekly_starting,number_of_sessions,average_session_length,number_of_unique_clients,tb_downloaded,tb_uploaded,cumulative_bandwidth,cumulative_sessions,cumulative_subscribers,sessions_per_user,GB_per_session,log_number_of_sessions,log_number_of_unique_clients,log_tb_downloaded,month
0,2020-01-05,23086410,00:04:07,683250,121.01,12.67,14353.83,2251215052,8461839,33.789111,0.005790,16.954755,13.434616,4.795873,1
1,2020-01-12,23584159,00:04:07,640118,116.52,11.14,14501.59,2284581863,8502485,36.843455,0.005413,16.976086,13.369408,4.758063,1
2,2020-01-19,22004022,00:04:07,592715,114.50,10.52,14626.62,2306585885,8536567,37.124119,0.005682,16.906736,13.292469,4.740575,1
3,2020-01-26,20245042,00:04:15,602685,118.05,10.13,14754.79,2326830927,8571523,33.591415,0.006331,16.823420,13.309150,4.771108,1
4,2020-02-02,22356460,00:04:11,617337,120.69,10.94,14886.42,2349187387,8615640,36.214353,0.005888,16.922626,13.333170,4.793225,2


### Let's define thresholds & derived columns


In [46]:
ENGAGED_GB_PER_SESSION = 0.035  # GB per session threshold for "engaged" / high-attention
# Activation thresholds: let's use 75th percentile of unique clients and sessions (volume-oriented)
users_75 = int(df['number_of_unique_clients'].quantile(0.75))
sessions_75 = int(df['number_of_sessions'].quantile(0.75))

In [47]:
print(f"Chosen engaged threshold (GB/session): {ENGAGED_GB_PER_SESSION}")
print(f"Activation thresholds (75th pct): users >= {users_75}, sessions >= {sessions_75}")

Chosen engaged threshold (GB/session): 0.035
Activation thresholds (75th pct): users >= 323924, sessions >= 5632312


#### Let's create new fields (`gb_used`, `heavy_usage_week`, `activated_week`) from the data:
- gb_used (week-level total GB) = GB_per_session * number_of_sessions
- heavy_usage_week (binary): GB_per_session >= ENGAGED_GB_PER_SESSION
- So a week is activated if both volume thresholds met

In [48]:
df['gb_used'] = df['GB_per_session'] * df['number_of_sessions']
df['heavy_usage_week'] = (df['GB_per_session'] >= ENGAGED_GB_PER_SESSION).astype(int)
df['activated_week'] = ((df['number_of_unique_clients'] >= users_75) &
                        (df['number_of_sessions'] >= sessions_75)).astype(int)

In [49]:
# equivalent count of engaged sessions (divide total GB by threshold).
df['engaged_session_equiv'] = df['gb_used'] / ENGAGED_GB_PER_SESSION

In [50]:
# how many engaged-equivalent sessions appear again next week?
df['engaged_next_week_equiv'] = df['engaged_session_equiv'].shift(-1)
df['repeat_equiv'] = df[['engaged_session_equiv', 'engaged_next_week_equiv']].min(axis=1)

#### Interpretations:
- engaged_session_equiv is an estimate not a count of sessions >= threshold (we don't have per-session sizes).
- repeat_equiv = min(current,next) is a conservative lower-bound proxy for persisted engaged volume.

### Cohorts Analysis & retention (weekly cohorts)
- Our cohort start where heavy_usage_week == 1 (Links 5G)

In [51]:
# Step 1: let't filter the data
cohort_starts = df.loc[df['heavy_usage_week'] == 1, date_col].sort_values().unique()

In [52]:
cohort_starts

<DatetimeArray>
['2022-05-08 00:00:00', '2022-05-15 00:00:00', '2022-05-22 00:00:00',
 '2022-05-30 00:00:00', '2022-06-06 00:00:00', '2022-06-13 00:00:00',
 '2022-06-26 00:00:00', '2022-07-03 00:00:00', '2022-07-10 00:00:00',
 '2022-08-14 00:00:00', '2022-11-27 00:00:00', '2023-01-15 00:00:00',
 '2023-01-22 00:00:00', '2023-01-29 00:00:00', '2023-02-05 00:00:00',
 '2023-02-12 00:00:00', '2023-02-19 00:00:00', '2023-02-26 00:00:00',
 '2023-03-05 00:00:00', '2023-03-12 00:00:00', '2023-03-19 00:00:00',
 '2023-03-26 00:00:00', '2023-06-11 00:00:00', '2023-07-09 00:00:00',
 '2023-07-23 00:00:00', '2023-07-30 00:00:00', '2023-08-06 00:00:00',
 '2023-08-27 00:00:00', '2023-10-01 00:00:00', '2023-11-19 00:00:00',
 '2023-11-26 00:00:00', '2024-01-14 00:00:00', '2024-01-21 00:00:00',
 '2024-01-28 00:00:00', '2024-02-04 00:00:00', '2024-02-11 00:00:00',
 '2024-02-18 00:00:00', '2024-02-25 00:00:00', '2024-03-03 00:00:00',
 '2024-03-10 00:00:00', '2024-03-17 00:00:00', '2024-03-24 00:00:00',
 '20

In [54]:
# Build cohort (one row per cohort start; columns W0, W1, W2, W3, W4)
cohort_rows = [] # an empty list to store the result where each row will be a dictionnary
for cw in cohort_starts:
    row = {'cohort_week': cw} # a dictionnary that hold each week
    for _ in range(0,5):  # W0..W4
        t = cw + pd.Timedelta(7*_, unit='d')
        selection = df[df[date_col] == t]
        row[f'W{_}'] = int(selection['heavy_usage_week'].iloc[0]) if not selection.empty else 0
    cohort_rows.append(row)

In [55]:
cohort_df = pd.DataFrame(cohort_rows).set_index('cohort_week')

In [56]:
cohort_df

,W0,W1,W2,W3,W4
cohort_week,,,,,
2022-05-08,1,1,1,0,0
2022-05-15,1,1,0,0,0
2022-05-22,1,0,0,0,0
2022-05-30,1,1,1,0,0
2022-06-06,1,1,0,0,0
...,...,...,...,...,...
2025-08-31,1,1,1,1,1
2025-09-07,1,1,1,1,0
2025-09-14,1,1,1,0,0


In [57]:
# Compute retention rates (for cohorts that start heavy: W0=1)
w1_rate = cohort_df['W1'].mean()
w4_rate = cohort_df[['W1','W2','W3','W4']].max(axis=1).mean()  # any heavy week within 4 weeks
print(f"W0->W1 retention: {w1_rate:.3%}")
print(f"W0->W4 return: {w4_rate:.3%}")

W0->W1 retention: 78.378%
W0->W4 return: 82.432%


Let's the heat map to visualize the retention rate

In [58]:
heat = cohort_df[['W0','W1','W2','W3','W4']].astype(int).T
fig_heat = px.imshow(heat, labels=dict(x="cohort week", y="weeks offset (W0 to W4)"),
                    x=[str(d.date()) for d in heat.columns], y=heat.index,
                    title="Cohort heatmap", text_auto=True)

In [59]:
fig_heat

## 2) RFM and Segmentation

In [60]:
WINDOW_WEEKS = 12 #we will segement on an intervalle of 12 weeks
rfm_records = [] # empty list to hold the each rfm row on the 12 weeks window
df_sorted = df.sort_values(date_col).reset_index(drop=True)

In [61]:
df_sorted.head()

,report_ending_weekly_starting,number_of_sessions,average_session_length,number_of_unique_clients,tb_downloaded,tb_uploaded,cumulative_bandwidth,cumulative_sessions,cumulative_subscribers,sessions_per_user,...,log_number_of_sessions,log_number_of_unique_clients,log_tb_downloaded,month,gb_used,heavy_usage_week,activated_week,engaged_session_equiv,engaged_next_week_equiv,repeat_equiv
0,2020-01-05,23086410,00:04:07,683250,121.01,12.67,14353.83,2251215052,8461839,33.789111,...,16.954755,13.434616,4.795873,1,133680.0,0,1,3.819429e+06,3.647429e+06,3.647429e+06
1,2020-01-12,23584159,00:04:07,640118,116.52,11.14,14501.59,2284581863,8502485,36.843455,...,16.976086,13.369408,4.758063,1,127660.0,0,1,3.647429e+06,3.572000e+06,3.572000e+06
2,2020-01-19,22004022,00:04:07,592715,114.50,10.52,14626.62,2306585885,8536567,37.124119,...,16.906736,13.292469,4.740575,1,125020.0,0,1,3.572000e+06,3.662286e+06,3.572000e+06
3,2020-01-26,20245042,00:04:15,602685,118.05,10.13,14754.79,2326830927,8571523,33.591415,...,16.823420,13.309150,4.771108,1,128180.0,0,1,3.662286e+06,3.760857e+06,3.662286e+06
4,2020-02-02,22356460,00:04:11,617337,120.69,10.94,14886.42,2349187387,8615640,36.214353,...,16.922626,13.333170,4.793225,2,131630.0,0,1,3.760857e+06,3.773714e+06,3.760857e+06


In [62]:
# For each week (as_of_week), compute R, F, M on prior WINDOW_WEEKS inclusive of that week
for idx, row in df_sorted.iterrows():
    as_of_week = row[date_col]
    start = as_of_week - pd.Timedelta(7*(WINDOW_WEEKS - 1), unit='d')
    window = df_sorted[(df_sorted[date_col] >= start) & (df_sorted[date_col] <= as_of_week)]
    if window.empty:
        continue
    # Recency: weeks since last heavy week in window (smaller = better). 
    heavy_weeks = window.loc[window['heavy_usage_week'] == 1, date_col]
    if heavy_weeks.empty:
        recency_weeks = np.nan
    else:
        recency_weeks = (as_of_week - heavy_weeks.max()).days / 7.0
    frequency = int(window['heavy_usage_week'].sum())  # count of heavy weeks in window
    monetary = float(window['gb_used'].sum())         # total GB in window (proxy)
    rfm_records.append({
        'as_of_week': as_of_week,
        'recency_weeks': recency_weeks,
        'frequency': frequency,
        'monetary_gb': monetary
    })

In [63]:
#let's create a dataframe of RFM
rfm_df = pd.DataFrame(rfm_records).dropna().reset_index(drop=True)

In [64]:
rfm_df.head()

,as_of_week,recency_weeks,frequency,monetary_gb
0,2022-05-08,0.0,1,1403470.0
1,2022-05-15,0.0,2,1380980.0
2,2022-05-22,0.0,3,1357200.0
3,2022-05-30,0.0,4,1219580.0
4,2022-06-06,0.0,5,1193550.0


In [65]:
# Convert to quartile scores:
rfm_df['R_score'] = pd.qcut(rfm_df['recency_weeks'].rank(method='first'), 4, labels=[4,3,2,1]).astype(int)
rfm_df['F_score'] = pd.qcut(rfm_df['frequency'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
rfm_df['M_score'] = pd.qcut(rfm_df['monetary_gb'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
rfm_df['RFM'] = rfm_df['R_score'].astype(str) + rfm_df['F_score'].astype(str) + rfm_df['M_score'].astype(str)

In [66]:
rfm_df.head()

,as_of_week,recency_weeks,frequency,monetary_gb,R_score,F_score,M_score,RFM
0,2022-05-08,0.0,1,1403470.0,4,1,1,411
1,2022-05-15,0.0,2,1380980.0,4,1,1,411
2,2022-05-22,0.0,3,1357200.0,4,2,1,421
3,2022-05-30,0.0,4,1219580.0,4,2,1,421
4,2022-06-06,0.0,5,1193550.0,4,2,1,421


In [67]:
# Segmentation rules
# le't create a simple function to apply 
def rfm_segment(r):
    if r['R_score'] >= 3 and r['F_score'] >= 3 and r['M_score'] >= 3:
        return 'Champions'
    if r['R_score'] <= 2 and r['F_score'] <= 2 and r['M_score'] <= 2:
        return 'Dormant'
    if r['F_score'] >= 3:
        return 'Frequent'
    if r['R_score'] <= 2 and r['F_score'] <= 2:
        return 'At-Risk'
    return 'Other'

In [68]:
rfm_df['segment'] = rfm_df.apply(rfm_segment, axis=1)

In [80]:
rfm_df.head(7)

,as_of_week,recency_weeks,frequency,monetary_gb,R_score,F_score,M_score,RFM,segment
0,2022-05-08,0.000000,1,1403470.0,4,1,1,411,Other
1,2022-05-15,0.000000,2,1380980.0,4,1,1,411,Other
2,2022-05-22,0.000000,3,1357200.0,4,2,1,421,Other
3,2022-05-30,0.000000,4,1219580.0,4,2,1,421,Other
4,2022-06-06,0.000000,5,1193550.0,4,2,1,421,Other
5,2022-06-13,0.000000,6,1173860.0,4,3,1,431,Frequent
6,2022-06-19,0.857143,6,1279840.0,3,3,1,331,Frequent


In [70]:
# Visual: Frequency vs Monetary, sized by Recency score, colored by segment
fig_rfm = px.scatter(rfm_df, x='frequency', y='monetary_gb', size='R_score', color='segment',
                        hover_data=['as_of_week', 'recency_weeks', 'RFM'],
                        title=f"RFM scatter (rolling {WINDOW_WEEKS}-week window)")

In [71]:
fig_rfm


### Interpretation & ROI:
- `Champions` are high recency/frequency/monetary -> best immediate ROI for premium sponsorships.
-  `Frequent` are reliable reach; 
- `At-Risk` or `Dormant` are candidate equity or infrastructure checks.

# Cohorts & Retention
- Cohorts built by week of first heavy_usage_week (the week where GB_per_session >= 0.035).
- For each cohort , I tracked W0 to W4 whether the week had heavy usage.
#### Retention results (city-level weekly cohorts)

- Number of cohorts that started with a heavy week: 74
- W0 → W1 retention (heavy weeks): 0.784 (78.4% of heavy-week cohorts had a heavy week the next calendar week)
- W0 → W4 return (had any heavy week in next 4 weeks): 0.824 (82.4%)

### Ethics & Equity (embedded)
Risk: if the City optimizes purely for high-yield kiosks (high GB/session or high sessions), investments and ad revenue will concentrate in already-well-served neighborhoods widening the digital divide.
- Measurement / bias: low usage areas are in risk of being labeled “low ROI”.

# RFM segmentation & ROI
Rolling 12-week window at city-level
- R (Recency): weeks since last heavy_usage_week within the rolling window. Lower recency → better score.
- F (Frequency): count of heavy_usage_week in the rolling 12-weeks window.
- M (Monetary proxy): total GB in the rolling 12-week window.
#### Scoring & segments
- Each of R, F, M is converted to quartile scores (1–4).
- Simple segmentation rules (examples used):
 - Champions: R ≥3, F ≥3, M ≥3 (top quartiles on all)
 - Dormant: low on all three
 - Frequent: high F
 - At-Risk: older recency & low frequency

#### Results
- Rows computed 154 as-of-week RFM observations
### RFM --> ROI
- Champions: best immediate ROI candidates for premium sponsorships (recent, frequent heavy usage, and lots of GB → higher attention/video-friendly).
- Frequent: dependable reach; good for mid-tier campaigns with frequency caps.
- At-Risk/Dormant: consider infrastructure checks, outreach, or equity programs rather than immediate monetization.
### Assumptions
- Monetary (GB) is a proxy for attention — higher GB usually correlates with longer viewing/interaction time but is not a perfect substitute for ad impressions or viewability metrics.



